In [1]:
import osmnx as ox
import networkx as nx
import pandas as pd
from vincenty import vincenty
import numpy as np
import lidar_elevation_coordinates_function as lid
import seaborn as sns

In [2]:
ROUTES = {}

In [3]:
def getBbox(l):
    min_lon = min([x[1] for x in l])
    max_lon = max([x[1] for x in l])
    min_lat = min([x[0] for x in l])
    max_lat = max([x[0] for x in l])
    
    return max_lat,min_lat,max_lon,min_lon

In [4]:
def getRoute(G, s_coord, e_coord, avoid = []):
    start_node = ox.get_nearest_node(G, s_coord)
    end_node = ox.get_nearest_node(G, e_coord)
    
    orig_lengths = []
    for item in avoid:
        print(item)
        orig_length = G.edges[(item[0], item[1], 0)]['length']
        orig_lengths.append(orig_length)
        G.edges[(item[0], item[1], 0)]['length'] = 50000
        
    route = nx.shortest_path(G, start_node, end_node, weight='length')
    length = nx.shortest_path_length(G, start_node, end_node, weight='length')
    
    for item in avoid:
        orig_length = orig_lengths.pop()
        print(item, orig_length)
        G.edges[(item[0], item[1], 0)]['length'] = orig_length
    
    return route, length

In [5]:
def getAllLocations(G,route):
    loc = []
    for node in route:
        loc.append((G.nodes[node]['y'],G.nodes[node]['x']))
    return loc

In [6]:
def getRouteAndAddToCollection(G, coord1, coord2, name, df, avoid = []):
    r, length = getRoute(G, coord1, coord2, avoid = avoid)
    l = getAllLocations(G, r)
    
    max_lat,min_lat,max_lon,min_lon = getBbox(l)
    
    p_50,p_100,p_250 = getNumAccPoi(df,l,min_lat,min_lon,max_lat,max_lon)
    
    e_down, e_up, distances, elevation, distrib = computeTotalElevation(l)

    ROUTES[name] = {'length':length,'locations':l,'route':r,'p50':p_50,'p100':p_100,'p250':p_250,'downhill':e_down,'uphill':e_up,'distances':distances, 'slope_distribution':distrib, 'elevation':elevation}

In [7]:
def computeTotalElevation(l):
    elev_down = 0
    elev_up = 0
    old_elev = -1
    distrib = []
    old_coord = -1
    
    distances = []
    
    elevation = []
    for item in l:
        _ = lid.get_elevation(item, '/shared/lidar/study_area.tif')
        diff = 0
        if _ is not None:
            if old_elev != -1:
                diff = (_[0]['elevation'] - old_elev)
                if diff < 0:
                    elev_down += diff
                else:
                    elev_up += diff
            old_elev = _[0]['elevation']
            
            if old_coord != -1:
                dist = vincenty(old_coord,item) * 1000
                distrib.append(diff/dist)
                distances.append(dist)
                elevation.append(_[0]['elevation'])

            
            old_coord = item
        

    return elev_down,elev_up,distances,elevation,distrib

In [8]:
def getNumAccPoi(df,l,lat_min,lon_min,lat_max,lon_max):
    _ = df[(df['lon']>lon_min-0.1)&(df['lon']<lon_max+0.1)&(df['lat']>lat_min-0.1)&(df['lat']<lat_max+0.1)]
    num_acc_pois_50  = 0
    num_acc_pois_100 = 0
    num_acc_pois_250 = 0
    for _row,_col in _.iterrows():
        _col['p50'] = False
        _col['p100'] = False
        _col['p250'] = False
        for _l in l:
            # Compute distance
            d = vincenty((_l[0],_l[1]),(_col['lat'],_col['lon'])) * 1000.0
            if d <= 50 and not _col['p50']:
                _col['p50'] = True
                num_acc_pois_50 += 1
            if d <= 100 and not _col['p100']:
                _col['p100'] = True
                num_acc_pois_100 += 1
            if d <= 250 and not _col['p250']:
                _col['p250'] = True
                num_acc_pois_250 += 1
    return num_acc_pois_50,num_acc_pois_100,num_acc_pois_250

In [9]:
G_walk = ox.graph_from_address(address="London", dist=5000, network_type="walk")
G_drive = ox.graph_from_address(address="London", dist=5000, network_type="drive")

In [10]:
#G = ox.graph.graph_from_bbox(51.58, 51.45, 0.08, -0.3, network_type='walk', simplify=True, retain_all=False, truncate_by_edge=False, clean_periphery=True, custom_filter=None)

#ox.plot_graph(G)

In [ ]:
df = pd.read_pickle('/shared/combined_accessible_pois.pkl')

# Define routes here
coord1 = (51.50815,-0.076303) # Tower of London
coord2 = (51.50457,-0.108711) # London Waterloo East Tube Station
coord3 = (51.50106,-0.124613) # Big Ben
coord4 = (51.48402,-0.606044) # Windsor castle

coord_tmp = (51.519599051500094, -0.15820421319589217)
coord_tmp_end = (51.49887208008789, -0.1048175303045337)

getRouteAndAddToCollection(G_drive,coord1,coord2,'short1-drive',df)
getRouteAndAddToCollection(G_drive,coord1,coord3,'short2-drive',df)
getRouteAndAddToCollection(G_drive,coord2,coord3,'short3-drive',df)

getRouteAndAddToCollection(G_walk,coord1,coord2,'short1',df)
getRouteAndAddToCollection(G_walk,coord1,coord3,'short2',df)
getRouteAndAddToCollection(G_walk,coord2,coord3,'short3',df)

getRouteAndAddToCollection(G_walk,coord1,coord2,'short1-avoid',df,avoid = [(ROUTES['short1']['route'][4],ROUTES['short1']['route'][5]),(ROUTES['short1']['route'][7],ROUTES['short1']['route'][8]),(ROUTES['short1']['route'][18],ROUTES['short1']['route'][19])])

getRouteAndAddToCollection(G_walk,coord_tmp,coord_tmp_end,'longer-route',df)

In [ ]:
dd = np.cumsum(ROUTES['longer-route']['distances'])
sns.lineplot(dd,ROUTES['longer-route']['elevation'])

In [ ]:
sns.lineplot(dd,ROUTES['longer-route']['slope_distribution'])


In [ ]:
routes = [ROUTES['short1-drive']['route'], ROUTES['short2-drive']['route'], ROUTES['short3-drive']['route']]
rc = ['r', 'y', 'c']
fig, ax = ox.plot_graph_routes(G_drive, routes, route_colors=rc, route_linewidth=8, node_size=4)


routes = [ROUTES['longer-route']['route'], ROUTES['short1']['route'], ROUTES['short3']['route']]
fig, ax = ox.plot_graph_routes(G_walk, routes, route_colors=rc, route_linewidth=8, node_size=4)

In [ ]:
bbox = getBbox(ROUTES['longer-route']['locations'])
fig, ax = ox.plot_graph_route(G_walk, ROUTES['longer-route']['route'], route_linewidth=8, node_size=4, bbox=bbox)

In [ ]:
l = getAllLocations(G_walk,ROUTES['longer-route']['route'])
with open('/shared/tmp_coord_longer.csv','w') as f:
    for i in l:
        f.write(str(i) + ';')